In [72]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/anyas-gojo-revival-space-shop/sample_submission.csv
/kaggle/input/anyas-gojo-revival-space-shop/test.tsv
/kaggle/input/anyas-gojo-revival-space-shop/train.tsv


In [73]:
import nltk
df=pd.read_csv("/kaggle/input/anyas-gojo-revival-space-shop/train.tsv",sep='\t')
df.head()

,date,time,id,reviewer name,reviewer address,contact number,variation,verified_reviews,sentiment,feedback
0,30-Jul-18,02:16:00,5138527,Theresa Diaz,"2373 Long Mews\nSouth Lorraine, FL 83715",7445448464,Black Plus,Love my new Echo Plus,5,1
1,30-Jul-18,20:13:24,7881409,Michael Wagner,"34120 James Well Apt. 930\nSouth Kenneth, AK 5...",(202)233-6662,Configuration: Fire TV Stick,Works really well.,5,1
2,29-Jul-18,03:20:48,5658396,Erin Rodriguez,"7996 Booth Mission\nTaylorport, AL 13457",(219)882-0705,Black Show,Echo Show is said to work with certain apps bu...,2,0
3,30-Jul-18,22:48:21,4317957,Christine Perez,USS Schaefer\nFPO AA 86432,291-900-6087,Heather Gray Fabric,Very cool product. Speaker sounds good with my...,5,1
4,23-Jul-18,12:36:42,7570569,Jamie Davila,"000 Kevin Cliffs Suite 674\nBrennanview, NM 77690",9527296610,Black Plus,So far I love it. It was easy to set up - if ...,5,1


In [74]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2520 entries, 0 to 2519
Data columns (total 10 columns):
 #   Column            Non-Null Count  Dtype 
---  ------            --------------  ----- 
 0   date              2520 non-null   object
 1   time              2520 non-null   object
 2   id                2520 non-null   int64 
 3   reviewer name     2520 non-null   object
 4   reviewer address  2520 non-null   object
 5   contact number    2520 non-null   object
 6   variation         2520 non-null   object
 7   verified_reviews  2519 non-null   object
 8   sentiment         2520 non-null   int64 
 9   feedback          2520 non-null   int64 
dtypes: int64(3), object(7)
memory usage: 197.0+ KB


In [75]:
df.isnull().sum()

date                0
time                0
id                  0
reviewer name       0
reviewer address    0
contact number      0
variation           0
verified_reviews    1
sentiment           0
feedback            0
dtype: int64

In [76]:
df=df.dropna()
df.isnull().sum()

date                0
time                0
id                  0
reviewer name       0
reviewer address    0
contact number      0
variation           0
verified_reviews    0
sentiment           0
feedback            0
dtype: int64

In [77]:
from sklearn.model_selection import train_test_split
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
import tensorflow as tf

y=df['sentiment']
X=df['verified_reviews']
y=y.apply(lambda x: 1 if x==5 else 0)

X_train,X_test,y_train,y_split = train_test_split(X,y,test_size=0.2,random_state=1)
X_train

1400                                   Really easy to use
1426    Love it except that Amazon needs a reference o...
1102    Although I haven't taken advantage of many of ...
1576    I like the fact that I can get any kind of mus...
440                             awesome thing, love Alexa
                              ...                        
960     this is so much fun to play with - I just ask ...
905                                                      
1096    Easy installation and set up. Better sound fro...
235     This thing is awesome for grocery lists, paire...
1061    It was supposed to come with a free Hue lightb...
Name: verified_reviews, Length: 2015, dtype: object

In [78]:
tokenizer = Tokenizer()
tokenizer.fit_on_texts(X_train)

In [79]:
# tokenizer.word_index

In [80]:
vocab_length = len(tokenizer.word_index)+1
vocab_length

3554

In [81]:
#convert review texts into sequences of integers
X_train = tokenizer.texts_to_sequences(X_train)
X_test = tokenizer.texts_to_sequences(X_test) 

In [82]:
# Find max sequence length
max_seq_length=np.max(list(map(lambda x:len(x),X_train)))
max_seq_length

555

In [83]:
#Pad sequences to be of uniform length
X_train = pad_sequences(X_train,maxlen=max_seq_length,padding='post')
X_test = pad_sequences(X_test,maxlen=max_seq_length,padding='post')
X_train

array([[ 75,  35,   3, ...,   0,   0,   0],
       [ 10,   4, 749, ...,   0,   0,   0],
       [519,   2, 298, ...,   0,   0,   0],
       ...,
       [ 35, 685,   5, ...,   0,   0,   0],
       [ 12, 107,   8, ...,   0,   0,   0],
       [  4,  31, 773, ...,   0,   0,   0]], dtype=int32)

In [84]:
X_train.shape

(2015, 555)

In [85]:
inputs = tf.keras.Input(shape=(X_train.shape[1],))
x = tf.keras.layers.Embedding(
 input_dim=vocab_length,
 output_dim=128,
 input_length=max_seq_length
)(inputs)

x=tf.keras.layers.GRU(256, return_sequences=True,activation='tanh')(x)

x=tf.keras.layers.Flatten()(x)
outputs = tf.keras.layers.Dense(1,activation='sigmoid')(x)

model=tf.keras.Model(inputs=inputs,outputs=outputs)

model.compile(
  optimizer='adam',
  loss='binary_crossentropy',
  metrics=['accuracy']
)

history = model.fit(
X_train,
y_train,
validation_split=0.2,
batch_size=32,
epochs=100,
callbacks=[
    tf.keras.callbacks.EarlyStopping(
      monitor='val_loss',
      patience=3,
      restore_best_weights=True
    )
])

Epoch 1/100
51/51 [==============================] - 12s 192ms/step - loss: 0.5926 - accuracy: 0.7134 - val_loss: 0.5707 - val_accuracy: 0.7270
Epoch 2/100
51/51 [==============================] - 7s 148ms/step - loss: 0.4114 - accuracy: 0.8269 - val_loss: 0.4605 - val_accuracy: 0.7940
Epoch 3/100
51/51 [==============================] - 6s 126ms/step - loss: 0.2620 - accuracy: 0.8970 - val_loss: 0.4663 - val_accuracy: 0.8089
Epoch 4/100
51/51 [==============================] - 6s 122ms/step - loss: 0.1930 - accuracy: 0.9293 - val_loss: 0.5080 - val_accuracy: 0.8040
Epoch 5/100
51/51 [==============================] - 5s 100ms/step - loss: 0.1446 - accuracy: 0.9485 - val_loss: 0.5133 - val_accuracy: 0.8040


In [86]:
inputs

<KerasTensor: shape=(None, 555) dtype=float32 (created by layer 'input_4')>

In [87]:
x

<KerasTensor: shape=(None, 142080) dtype=float32 (created by layer 'flatten_3')>

In [88]:
model.evaluate(X_test,y_split)

16/16 [==============================] - 0s 12ms/step - loss: 0.3953 - accuracy: 0.8433


[0.39525532722473145, 0.8432539701461792]

In [89]:
model.summary()

Model: "model_3"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_4 (InputLayer)        [(None, 555)]             0         
                                                                 
 embedding_3 (Embedding)     (None, 555, 128)          454912    
                                                                 
 gru_3 (GRU)                 (None, 555, 256)          296448    
                                                                 
 flatten_3 (Flatten)         (None, 142080)            0         
                                                                 
 dense_3 (Dense)             (None, 1)                 142081    
                                                                 
Total params: 893441 (3.41 MB)
Trainable params: 893441 (3.41 MB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


In [105]:
y_pred=model.predict(X_test)
y_pred
# y_split

16/16 [==============================] - 0s 20ms/step


array([[8.2710582e-01],
       [8.3970571e-01],
       [9.2668104e-01],
       [9.8175186e-01],
       [6.8260157e-01],
       [9.4406390e-01],
       [5.0505990e-01],
       [1.3795616e-01],
       [9.6342480e-01],
       [7.3462528e-01],
       [9.5515990e-01],
       [5.8983469e-01],
       [7.6498580e-01],
       [9.1160226e-01],
       [3.2419655e-01],
       [5.9200954e-01],
       [1.6297258e-01],
       [9.7600752e-01],
       [9.7564358e-01],
       [9.0944541e-01],
       [9.7195971e-01],
       [9.7323281e-01],
       [9.7919238e-01],
       [9.6512914e-01],
       [7.1388823e-01],
       [9.4564694e-01],
       [8.6710715e-01],
       [8.3134741e-01],
       [1.1992873e-01],
       [9.5270938e-01],
       [9.1209710e-01],
       [9.7350794e-01],
       [1.3149188e-01],
       [9.7632116e-01],
       [9.6822637e-01],
       [8.9544082e-01],
       [2.7155459e-02],
       [8.9676809e-01],
       [8.7095922e-01],
       [2.6942706e-01],
       [9.5341939e-01],
       [1.956634

In [107]:
from sklearn.metrics import f1_score
f1_score(y_split,y_pred,average='binary')

/opt/conda/lib/python3.10/site-packages/sklearn/utils/validation.py:605: FutureWarning: is_sparse is deprecated and will be removed in a future version. Check `isinstance(dtype, pd.SparseDtype)` instead.
  if is_sparse(pd_dtype):
/opt/conda/lib/python3.10/site-packages/sklearn/utils/validation.py:614: FutureWarning: is_sparse is deprecated and will be removed in a future version. Check `isinstance(dtype, pd.SparseDtype)` instead.
  if is_sparse(pd_dtype) or not is_extension_array_dtype(pd_dtype):
/opt/conda/lib/python3.10/site-packages/sklearn/utils/validation.py:605: FutureWarning: is_sparse is deprecated and will be removed in a future version. Check `isinstance(dtype, pd.SparseDtype)` instead.
  if is_sparse(pd_dtype):
/opt/conda/lib/python3.10/site-packages/sklearn/utils/validation.py:614: FutureWarning: is_sparse is deprecated and will be removed in a future version. Check `isinstance(dtype, pd.SparseDtype)` instead.
  if is_sparse(pd_dtype) or not is_extension_array_dtype(pd_dtyp

ValueError: Classification metrics can't handle a mix of binary and continuous targets

In [91]:
#importing test datset
test_df=pd.read_csv("/kaggle/input/anyas-gojo-revival-space-shop/test.tsv",sep='\t')
test_df.head()

,date,time,id,reviewer name,reviewer address,contact number,variation,verified_reviews,feedback
0,29-Jul-18,16:47:11,6074908,Russell Hensley,"9059 Morales Run Suite 912\nSouth Charles, MH ...",6932290613,Heather Gray Fabric,I have the original Alexa and the Tap and so f...,1
1,28-Jul-18,17:49:58,1159008,Adam Baxter,"13728 Jones Mission Apt. 526\nSouth Tylerfurt,...",385-994-9458,Black Plus,Great product. More skills are needed. I love it.,1
2,30-Jul-18,16:46:29,1759753,Sarah Contreras,"7113 Pratt Plains\nPalmermouth, IN 02370",(738)354-8387,Configuration: Fire TV Stick,I will never buy anything Amazon makes again!T...,0
3,29-Jul-18,19:23:32,1861422,Tim Castillo,"93586 Cook Union Apt. 080\nMorganmouth, WI 51175",(294)877-6223,Black Plus,Not perfect but very good! A few times Alexa p...,1
4,28-Jul-18,16:15:25,3116018,Barry Young,69809 Harrington Meadows Apt. 256\nLake Courtn...,262-863-1391,Black Show,Kids like it for games. Showing the lyrics fo...,1


In [92]:
test_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 630 entries, 0 to 629
Data columns (total 9 columns):
 #   Column            Non-Null Count  Dtype 
---  ------            --------------  ----- 
 0   date              630 non-null    object
 1   time              630 non-null    object
 2   id                630 non-null    int64 
 3   reviewer name     630 non-null    object
 4   reviewer address  630 non-null    object
 5   contact number    630 non-null    object
 6   variation         630 non-null    object
 7   verified_reviews  630 non-null    object
 8   feedback          630 non-null    int64 
dtypes: int64(2), object(7)
memory usage: 44.4+ KB


In [93]:
x=test_df['verified_reviews']
x

0      I have the original Alexa and the Tap and so f...
1      Great product. More skills are needed. I love it.
2      I will never buy anything Amazon makes again!T...
3      Not perfect but very good! A few times Alexa p...
4      Kids like it for games.  Showing the lyrics fo...
                             ...                        
625    Best thing I have ever owned! Started with the...
626    I love being able to control lights, doors and...
627                                          works great
628    Purchased for my 83 year old mother-she loves ...
629         Fun so far...still learning how it all works
Name: verified_reviews, Length: 630, dtype: object

In [70]:
# tokenizer = Tokenizer()
# tokenizer.fit_on_texts(x)


In [94]:
x = tokenizer.texts_to_sequences(x) 
# x

In [61]:
# max_length=np.max(list(map(lambda t:len(t),x)))
# max_length

232

In [95]:
x = pad_sequences(x,maxlen=max_seq_length,padding='post')
x

array([[  2,  15,   1, ...,   0,   0,   0],
       [ 16,  54,  40, ...,   0,   0,   0],
       [  2,  69, 274, ...,   0,   0,   0],
       ...,
       [ 33,  16,   0, ...,   0,   0,   0],
       [190,   9,   7, ...,   0,   0,   0],
       [ 94,  22,  98, ...,   0,   0,   0]], dtype=int32)

In [97]:
y = model.predict(x)
y
# x.shape

20/20 [==============================] - 0s 18ms/step


array([[6.73306108e-01],
       [9.13103223e-01],
       [3.83143634e-01],
       [1.86773650e-02],
       [8.41242671e-01],
       [6.73306108e-01],
       [9.49145019e-01],
       [9.68805730e-01],
       [2.03165740e-01],
       [5.54150701e-01],
       [8.27105820e-01],
       [9.86433625e-01],
       [8.42754483e-01],
       [7.24545479e-01],
       [9.89498854e-01],
       [9.48323905e-01],
       [3.92814904e-01],
       [9.88472879e-01],
       [1.89430192e-02],
       [1.71098858e-02],
       [9.69829381e-01],
       [9.43853438e-01],
       [8.79142940e-01],
       [6.84768379e-01],
       [9.03903604e-01],
       [9.80005980e-01],
       [1.00320943e-01],
       [6.08541667e-01],
       [8.68825257e-01],
       [2.67563254e-01],
       [8.70069683e-01],
       [6.69550240e-01],
       [9.11850333e-01],
       [8.25201333e-01],
       [9.75343347e-01],
       [8.86017680e-01],
       [8.64071846e-01],
       [9.49722052e-01],
       [8.66789639e-01],
       [9.89033818e-01],


##### 

In [104]:
data = {'id': test_df['id'].values, 'target': y.flatten()}
submission_df = pd.DataFrame(data)
submission_df
# Export to CSV
# submission_df.to_csv('submission.csv', index=False)

,id,target
0,6074908,0.673306
1,1159008,0.913103
2,1759753,0.383144
3,1861422,0.018677
4,3116018,0.841243
...,...,...
625,6856853,0.988701
626,774782,0.981232
627,1194506,0.932881
628,4145653,0.936504
